# Introduction

This notebook is designed to extract information of registration and banking status for each cell line from Cellosaurus

# Set Up

In [2]:
# set up
from google.colab import drive
drive.mount('/content/drive')

%run '/content/drive/My Drive/hPSC-FAIRness Analysis/scripts/setup_drive.py'

root_dir, data_dir, processed_dir, results_dir = setup_drive()

Mounted at /content/drive
Mounted at /content/drive
Setting up root directory with name: 'hPSC-FAIRness Analysis'
Root directory path: '/content/drive/My Drive/hPSC-FAIRness Analysis'


## Load Cellosaurus hPSC

In [3]:
df = pd.read_excel(os.path.join(processed_dir,'hPSC Cellosaurus.xlsx'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21674 entries, 0 to 21673
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   AC      21674 non-null  object
 1   ID      21674 non-null  object
 2   SY      15411 non-null  object
 3   DR      21627 non-null  object
 4   RX      11675 non-null  object
 5   CC      21653 non-null  object
 6   OX      21674 non-null  object
 7   HI      5341 non-null   object
 8   CA      21674 non-null  object
 9   DT      21674 non-null  object
 10  WW      2144 non-null   object
 11  SX      20936 non-null  object
 12  AG      20178 non-null  object
 13  DI      10510 non-null  object
 14  ST      1213 non-null   object
 15  OI      6814 non-null   object
 16  AS      35 non-null     object
dtypes: object(17)
memory usage: 2.8+ MB


In [4]:
# the number of ESC and iPSC
# RESULT - hESCs: 4,763; hiPSCs: 16,911
category_counts = df['CA'].value_counts()
print(category_counts)
print(df.shape)

CA
Induced pluripotent stem cell    16911
Embryonic stem cell               4763
Name: count, dtype: int64
(21674, 17)


# 1. Assign Country to hPSCs

In this step, we will write a function to extract the associated country for each cell line. The function will then create a new column, **'Country'**, in the DataFrame to store this information.

In [5]:
import ast

def extract_country(input_data):
  if isinstance(input_data,str):
    # Check if the input_data contains a list representation
    if '[' in input_data and ']' in input_data:
      # Use ast.literal_eval() to safely evaluate the string representation to a list
        input_list = ast.literal_eval(input_data)
        # Process the list if it starts with 'From:'
        for line in input_list:
          if isinstance(line, str) and line.startswith('From:'):
              parts = line.split(';')
              return parts[-1].strip().rstrip('.')

        return 'Unknown'
        # Process the string if it starts with 'From:'
    elif input_data.startswith('From:'):
      parts = input_data.split(';')
      return parts[-1].strip().rstrip('.')

    return 'Unknown'

In [6]:
# Apply the function to the 'CC' column and create a new column 'Country'
df['Country'] = df['CC'].apply(extract_country)

# categorize results
country_counts = df['Country'].value_counts(dropna=False)

# fill NA wit 'Unknown'
df['Country'] = df['Country'].fillna('Unknown')
print(df['Country'].value_counts(dropna=False))


Country
Unknown                 7525
USA                     5301
United Kingdom          2161
China                   1305
Germany                 1063
Japan                    433
Spain                    361
South Korea              317
Australia                316
France                   236
Netherlands              232
Italy                    216
Israel                   199
Denmark                  197
Sweden                   177
Russia                   156
Ireland                  145
Taiwan                   142
Iran                     127
India                    126
Brazil                   115
Canada                   105
Saudi Arabia              88
Belgium                   72
Thailand                  66
Switzerland               65
Finland                   65
Poland                    47
Austria                   41
Czech Republic            39
Portugal                  37
Jordan                    36
Quatar                    34
Luxembourg                24
Turkey

**Note**

Since there are a lots of lines with country 'unkown', we will have some ways to resue the country information in another analysis

# 2. Assign Registration and Banking Status

In this step, new columns for all banks and registries will be created. Each cell line will have a Boolean value (`yes` or `nan`) in these columns, indicating whether the cell line is associated with a bank, a registry, or both.


## Step 1: Assign **All** infrastructures where each lines can be found

- create columns with all infrastructures, with value `yes` meaning the the cell line can be found in those infrastructures

In [7]:
# Write a function that extracts and returns the infrastructures (registries or biobanks) each hPSC line is registered or banked in Cellosaurus
def extract_registry_bank(df):
  # create a set to store unique infrastructure entries
  Infrastructure_set = set()

  # for each row in the dataframe
  for index in df.index:
    # get the value from DR column
    data = df.loc[index, 'DR']
    # Step 1. Extract the unique bank/registry names associated with each cell line
    Infrastructure_list = []

    if isinstance(data,str):
      # Check if the input_data contains a list representation
      if '[' in data and ']' in data:
        # Use ast.literal_eval() to safely evaluate the string representation to a list
        input_list = ast.literal_eval(data)

        # extract the registry/bank name
        for line in input_list:
          parts = line.split(';')
          Infrastructure_list.append(parts[0])

          Infrastructure_set.add(parts[0])
      else:
        parts = data.split(';')
        Infrastructure_list.append(parts[0])
        Infrastructure_set.add(parts[0])

    # Step 2. Create new columns for each unique bank/registry name
    for i in Infrastructure_list:
      df.loc[index, i] = 'yes'

  return df, Infrastructure_set

In [8]:
# apply to my dataframe
df,Infrastructure_set = extract_registry_bank(df)
print(Infrastructure_set)
print(len(Infrastructure_set))

{'WiCell', 'BCRC', 'ENCODE', 'NHCDR', 'Ubigene', 'BioSamples', 'CCRID', 'ATCC', 'RSCB', 'Kerafast', 'Wikidata', 'RCB', 'Sigma-Aldrich', 'GEO', 'CLO', 'ArrayExpress', 'IGRhCellID', '4DN', 'NIHhESC', 'TOKU-E', 'BioSample', 'EFO', 'JCRB', 'BioGRID_ORCS_Cell_line', 'EBiSC', 'SKIP', 'BTO', 'hPSCreg', 'ECACC', 'Millipore', 'EGA', 'CancerTools', 'HipSci', 'Coriell', 'TNGB', 'FCDI', 'Ximbio', 'LINCS_LDP', 'CLS', 'PRIDE', 'Lonza'}
41


- Occurance of each infrastructures for Cellosaurus hPSCs

In [9]:
# Count the frequency of each Infrastructure
counts = {}
for col in Infrastructure_set:
  count_yes = df[col].eq('yes').sum()
  counts[col] = count_yes

# Sort the frequency dictionary in order
sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
for key, value in sorted_counts.items():
  print(key,value)

Wikidata 21458
hPSCreg 9860
BioSamples 7019
SKIP 4916
RCB 2174
Coriell 1964
ECACC 1772
FCDI 1618
WiCell 1504
EBiSC 1223
HipSci 1221
GEO 792
NIHhESC 503
ArrayExpress 460
NHCDR 338
RSCB 123
BCRC 99
BioSample 97
EGA 76
LINCS_LDP 70
EFO 58
ENCODE 52
PRIDE 32
CLO 31
JCRB 23
ATCC 20
IGRhCellID 20
CCRID 13
CLS 12
Ubigene 10
4DN 4
TOKU-E 4
BTO 4
BioGRID_ORCS_Cell_line 3
Millipore 3
Kerafast 2
Ximbio 2
Sigma-Aldrich 1
CancerTools 1
TNGB 1
Lonza 1


## Step 2: Create a Lookup Table for All Infrastructures

In this step, we will create a lookup table containing detailed information about various stem cell infrastructures. This table will help organize and categorize the infrastructures for better understanding and analysis.

**Details to include:**
1. **Abbreviation:** The abbreviation used in Cellosaurus for the infrastructure.
2. **Name:** The full name of the infrastructure.
3. **Category:** Classification of the infrastructure based on its purpose. The categories are:
   - **Registry:** A dedicated stem cell registry.
   - **Bank:** A dedicated stem cell bank.
   - **Other bank or registry:** An institution where stem cell lines are a small proportion or not the primary focus.
   - **Others:** Databases that don’t fall into the categories above.

*Note:* Our definitions of stem cell registries and banks differ from those in Cellosaurus. In our context, we focus on non-profit registries and banks that either exclusively contain human pluripotent stem cells (hPSCs) or have stem cells as a major focus. Therefore, we will create a separate column to highlight this selection criterion.

4. **Cellosaurus Category:** The infrastructure's category as listed in the Cellosaurus database. You can find this information in the [Cellosaurus_xrefs.txt](https://ftp.expasy.org/databases/cellosaurus/cellosaurus_xrefs.txt).
5. **Country/Region:** The country or region where the infrastructure is based.
6. **Occurrence:** The frequency or occurrence of the infrastructure in Cellosaurus' hPSC records.
7. **URL:** The current URL of the infrastructure.
8. **Previous Website (if retired):** If the infrastructure has been retired or its website changed, include additional URLs (e.g., from the Wayback Machine) to provide historical information.

By compiling this information, we generated **Infrastructure Lookup Table.xlsx**.


### Step 3: Filter for Stem Cell Banks and Registries

In this step, we will filter our infrastructure columns in our Dataframe to include only stem cell bank and registry based on the information in the **Infrastructure Lookup Table.xlsx**, as these are the key focus of our analysis. Additionally, we will create three new columns to indicate whether a cell lin is banked, registered, or either banked or registered.

**Logic for the new columns:**
- **Banked:** A line is considered banked if it appears in any stem cell bank.
- **Registered:** A cell line is considered registered if it appears in any stem cell registry.
- **Registered/Bank:** A cell line is considered either banked or registered if it appears in any stem cell bank or registry.

These new columns will help categorize and identify which cell lines have been banked, registered, or both, facilitating a more focused analysis.

In [10]:
# Load Infrastructure Lookup Table
lookup_df = pd.read_excel(os.path.join(results_dir, 'Infrastructure Lookup Table.xlsx'))
lookup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Abbreviation                   41 non-null     object
 1   Name                           41 non-null     object
 2   Category                       41 non-null     object
 3   Cellosaurus Category           41 non-null     object
 4   Country/Region                 27 non-null     object
 5   Occurance                      41 non-null     int64 
 6   URL                            41 non-null     object
 7   Previous Website (if retired)  2 non-null      object
dtypes: int64(1), object(7)
memory usage: 2.7+ KB


### Filter columns

To only include stem cell banks and registries rather than all type of infrastructures in out dataframe

In [11]:
# Filter the columns in df based on 'Category' column in lookup_df
filtered_columns = lookup_df.loc[lookup_df['Category'].isin(['Bank', 'Registry']), 'Abbreviation']

# Select only the columns in df that match the filtered_columns
filtered_df = df[['AC', 'ID', 'CA', 'DR', 'DT', 'OX', 'CC', 'SX', 'AG', 'SY', 'RX', 'DI', 'OI', 'HI', 'WW', 'ST', 'AS','Country'] + filtered_columns.tolist()]

#print(filtered_df.head())

### Create new columns

For registration and banking status

In [12]:
# create a lookup dictonary for reference
lookup_dict = lookup_df.set_index('Abbreviation')['Category'].to_dict()
print(lookup_dict)

{'GEO': 'Others', 'BTO': 'Others', 'BioSamples': 'Others', 'BioSample': 'Others', 'ENCODE': 'Others', '4DN': 'Others', 'RCB': 'Bank', 'Coriell': 'Bank', 'ECACC': 'Bank', 'FCDI': 'Bank', 'WiCell': 'Bank', 'EBiSC': 'Bank', 'NHCDR': 'Bank', 'RSCB': 'Bank', 'BCRC': 'Bank', 'JCRB': 'Bank', 'ATCC': 'Bank', 'CLS': 'Company', 'Ubigene': 'Company', 'Millipore': 'Company', 'Kerafast': 'Company', 'Ximbio': 'Other bank', 'Sigma-Aldrich': 'Company', 'CancerTools': 'Company', 'TNGB': 'Other bank', 'hPSCreg': 'Registry', 'SKIP': 'Registry', 'HipSci': 'Bank', 'NIHhESC': 'Registry', 'LINCS_LDP': 'Database', 'CLO': 'Others', 'IGRhCellID': 'Others', 'CCRID': 'Database', 'TOKU-E': 'Database', 'Lonza': 'Others', 'BioGRID_ORCS_Cell_line': 'Others', 'Wikidata': 'Others', 'EFO': 'Others', 'ArrayExpress': 'Others', 'PRIDE': 'Others', 'EGA': 'Others'}


In [13]:
# function to determine if a cell line is registered
# A line is considered banked if it appears in any stem cell bank.
def is_registered(row):
  for col in row.index:
    if col in lookup_dict and lookup_dict[col] == 'Registry' and row[col] =='yes':
      return True
  return False

# function to determine if a cell line is banked
# A cell line is considered registered if it appears in any stem cell registry.
def is_banked(row):
    for col in row.index:
        if col in lookup_dict and lookup_dict[col] == 'Bank' and row[col] =='yes':
          return True
    return False

# function to determine if a cell line is either registered or banked
# A cell line is considered either banked or registered if it appears in any stem cell bank or registry.
def registered_or_banked(row):
    for col in row.index:
        if col in lookup_dict and (lookup_dict[col] == 'Bank' or 'Registry') and row[col] =='yes':
            return True
    return False

In [14]:
test = filtered_df.iloc[:5,-20:].copy()
filtered_df['Registered'] = test.apply(is_banked, axis=1)


<ipython-input-14-6187adfbd0cd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Registered'] = test.apply(is_banked, axis=1)


In [15]:
test

,HI,WW,ST,AS,Country,RCB,Coriell,ECACC,FCDI,WiCell,EBiSC,NHCDR,RSCB,BCRC,JCRB,ATCC,hPSCreg,SKIP,HipSci,NIHhESC
0,NaN,NaN,NaN,NaN,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,yes,nan,nan
1,NaN,NaN,NaN,NaN,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,yes,nan,nan
2,CVCL_A2TJ ! 03-01 fibroblast,NaN,NaN,NaN,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,CVCL_A2TJ ! 03-01 fibroblast,NaN,NaN,NaN,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,CVCL_A2TJ ! 03-01 fibroblast,NaN,NaN,NaN,Unknown,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [16]:
# add a new column to store the registration status
filtered_df['Registered'] = filtered_df.apply(is_registered, axis=1)
# add a new column to store the banking status
filtered_df['Banked'] = filtered_df.apply(is_banked, axis=1)
# add a new column to store either banking or registration status
filtered_df['Registered or Banked'] = filtered_df.apply(registered_or_banked, axis=1)

<ipython-input-16-4000acb348a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Registered'] = filtered_df.apply(is_registered, axis=1)
<ipython-input-16-4000acb348a2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Banked'] = filtered_df.apply(is_banked, axis=1)
<ipython-input-16-4000acb348a2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

# Assign publication Status

In [18]:
filtered_df['Published'] = filtered_df['RX'].apply(lambda x: False if pd.isna(x) or x == '' else True)

# save result
filtered_df.to_excel(os.path.join(processed_dir, 'Registration & Banking Status.xlsx'), index=False)

<ipython-input-18-5eae468990c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Published'] = filtered_df['RX'].apply(lambda x: False if pd.isna(x) or x == '' else True)
